<img src="https://rasa.com/docs/nlu/_static/rasa_logo.svg" width="200px" />

[CORE src](https://github.com/RasaHQ/rasa_core)
;
[CORE docs](https://rasa.com/docs/core/)
;
[NLU src](https://github.com/RasaHQ/rasa_nlu)
;
[NLU docs](https://rasa.com/docs/nlu/)

# 1. Setup

If you already have Rasa and the spacy english language model on your system (i.e using profided container) you can skip ahead.  

## 1.1 Install dependencies

I'm aware that at the moment of writing this `rasa_core` is at `v0.12.1` but it seems that the latest version has something broken inside `Agent#train` making it to not read the passed args properly.  
So for the sake of speed we're using an older version. ¯\\_(ツ)_/¯

In [ ]:
import sys
python = sys.executable

# install the rasa nlu and actions dialog
!{python} -m pip install -U rasa_core==0.9.0a7 rasa_nlu[spacy]==0.12.0;

# install a language model for spacy:
!{python} -m spacy download en_core_web_md;
!{python} -m spacy link --force en_core_web_md en;

  Using cached https://files.pythonhosted.org/packages/f1/c9/ab1ad7e342abb762398ca7780aa477f7c043edb3709e8ec77498c5396a48/rasa_core-0.9.0a7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/38/3b/b6765d15c8d14d844b754f8693a2430fee72c06eb537ca44d31b8c783955/rasa_nlu-0.13.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/70/203660597d12788e958dd691aa11c3c29caa075eadb2ce94d2eb53099d1b/mattermostwrapper-2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1d/c2/3f74cbd68fcbb9680c4bf6434f3c08c9d90a0ddd1e94b954db7382264ef7/fbmessenger-5.3.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/91/55/8cb23a97301b177e9c8e3226dba45bb454411de2cbd25746763267f226c2/tqdm-4.28.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.or

## 1.2 Check that everything is in place

You should be able to see the libs in the current context and load the spacy `en` language model.

In [1]:
import rasa_nlu
import rasa_core
import spacy

print('rasa_core: {}'.format(rasa_core.__version__))
print('rasa_nlu: {}'.format(rasa_nlu.__version__))
print('spacy: {}'.format(spacy.__version__))
print('spacy en model: {}'.format(spacy.load('en')('Hello world!')))

rasa_core: 0.9.0a7
rasa_nlu: 0.12.0
spacy: 2.0.16
spacy en model: Hello world!


# 2. Configuration

## 2.1 Natural language understanding

For this purpouse we need to create sets of `intent`s.  
The intent describes what the messages *mean*. [More information about the data format](https://nlu.rasa.com/dataformat.html#markdown-format).  
  
This is the training data for the `NLU` model, one example per line.  
Entities are labeled using the markdown link syntax: `[entity value](entity_type)`:

In [2]:
nlu_md = '''
## intent:greet
- hi
- hey
- hello
- good morning
- good evening
- good afternoon

## intent:goodbye
- bye
- bye bye
- goodbye
- good bye
- good night
- see you
- see you later
- see you around
- have a nice day

## intent:mood_affirm
- yes
- indeed
- correct
- of course
- that sounds good

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- ok
- fine
- good
- perfect
- very good
- extremely good
- great
- amazing
- wonderful
- super
- I am feeling very good
- I am great

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- so sad

## intent:weather
- weather [London](LOCATION)                          <!-- see how to define generic entities, current locations are hardcoded :| -->
- weather in [Iasi](LOCATION)
- what is the weather in [Paris](LOCATION)            <!-- isn't there any regex-like synthax to avoid writing different entries for short form?! -->
- what's the weather in [Cluj](LOCATION)
- how is the weather in [Bucharest, Romania](LOCATION)
- how's the weather in [berlin](LOCATION)
'''

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


To properly train the NLU model, you also need to define what is part of that model.  
Rasa NLU uses a similar pipeline concept as sklearn does.  
All the components that are listed in the pipeline will be trained one after another and everyone of them contributes its part to the structured data extraction:

In [3]:
config = '''
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_spacy"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
'''

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


# 3. Training

## 3.1 Train the NLU model

Train a model to recognise the `intent`s defined above, so that a message like `"hello"` will recognised this as a `"greet"` intent.

In [4]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data('nlu.md')

# initialize trainer
trainer = Trainer(config.load('config.yml'))

# train the model
interpreter = trainer.train(training_data)

# persist model
model_directory = trainer.persist('./models/nlu', fixed_model_name = 'current')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

Test that the model is properly trained with a simple `"hello"`.

In [5]:
print(interpreter.parse('hello'))

{'intent': {'name': 'greet', 'confidence': 0.676677705182363}, 'entities': [], 'intent_ranking': [{'name': 'greet', 'confidence': 0.676677705182363}, {'name': 'goodbye', 'confidence': 0.16754670923232573}, {'name': 'mood_affirm', 'confidence': 0.0647799130336675}, {'name': 'mood_great', 'confidence': 0.04171662225870841}, {'name': 'mood_unhappy', 'confidence': 0.021851854763845702}, {'name': 'weather', 'confidence': 0.020719775862290356}, {'name': 'mood_deny', 'confidence': 0.006707419666800021}], 'text': 'hello'}


## 3.2 Train dialog model

### 3.2.1 Stories

A story starts with `##` and you can give it a name.  
  
Lines that start with `*` are messages sent by the user. Although you don't write the *actual* message, but rather the intent (and the entities) that represent what the user *means*. If you don't know about intents and entities, don't worry! We will talk about them more later. 
  
Lines that start with `-` are *actions* taken by your bot. In this case all of our actions are just messages sent back to the user, like `utter_greet`, but in general an action can do anything, including calling an API and interacting with the outside world. 

In [6]:
stories_md = '''
## happy path              <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## sad path affirm
* greet
  - utter_greet
* mood_unhappy
  - utter_unsplash
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path deny
* greet
  - utter_greet
* mood_unhappy
  - utter_unsplash
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_sorry
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## weather
* weather
  - utter_weather

## goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear
'''

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


### 3.2.2 Domain

The domain specifies the universe that the bot lives in.  
You should list all of the intents and actions that show up in your stories.  
This is also the place to write templates, which contain the messages the bot can send back.  

In [7]:
domain_yml = '''
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- weather

slots:
  unsplash_image_url:
    type: text

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_sorry
- utter_unclear
- __main__.UnsplashAction
- __main__.WeatherAction

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up: {unsplash_image_url}"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_sorry:
  - text: "Sorry to hear that, go find a hug!"

  utter_goodbye:
  - text: "Bye"
  
  utter_weather:
  - text: "Not used"
'''

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### 3.2.3 Custom Actions

Sometimes, you not only want to send back messages to the user, but you also want to call an API or run some code.  
You can create custom actions that will be called once the bots ML model predicts them.  

### 3.2.3.1 Unsplash

Get cat images to cheer somebody up.

In [8]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet

class UnsplashAction(Action):
    
    def name(self):
        return 'utter_unsplash'

    def run(self, dispatcher, tracker, domain):
        url = 'https://source.unsplash.com/1600x900/?cat'
        return [SlotSet("unsplash_image_url", url)]  

### 3.2.3.2 Weather

In [9]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet

import requests

class WeatherAction(Action):
    
    def name(self):
        return "utter_weather"

    def run(self, dispatcher, tracker, domain):
        location = next(tracker.get_latest_entity_values('GPE'), None)
        query = 'select * from weather.forecast where woeid in (select woeid from geo.places(1) where text="{}") and u="c"'.format(location)
        url = 'https://query.yahooapis.com/v1/public/yql?format=json&q={}'.format(query)
        
        response = requests.get(url)
        
        if response.status_code == 200:
            item = response.json()['query']['results']['channel']['item']
            dispatcher.utter_message("It's {condition} and {temperature}°C in {location}".format(**{
                'condition': item['condition']['text'],
                'temperature': item['condition']['temp'],
                'location': item['title'].replace('Conditions for ', '')
            }))
        else:
            dispatcher.utter_message('Could not retrieve weather for "{}"'.format(location))
            
        return [] # see how to pass slots from here, like with unsplash  

### 3.2.4 Train Model

In [10]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(
    fallback_action_name = 'utter_unclear',
    core_threshold = 0.2,
    nlu_threshold = 0.6
)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# load defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split = 0.0,
    epochs = 400
)

agent.persist('models/dialogue')

Using TensorFlow backend.
Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 17.06it/s, # trackers=20]
Processed actions: 245it [00:02, 92.75it/s, # examples=245]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 19)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6656      
_________________________________________________________________
dense_1 (Dense)              (None, 11)                363       
_________________________________________________________________
activation_1 (Activation)    (None, 11)                0         
Total params: 7,019
Trainable params: 7,019
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
245/245 [==============================] - 1s 4ms/step - loss: 2.3220 - acc: 0.2612
Epoch 2/400
245/245 [==============================] - 0s 917us/step - loss: 2.2209 - acc: 0.4327
Epoch 3/400
245/245 [==============================] - 0s 1ms/step - loss: 2.1

# 4. Testing

Now that models are trained we can instantiate an `Agent` and query it.

In [11]:
from rasa_core.agent import Agent

agent = Agent.load(
    'models/dialogue', 
    interpreter = model_directory
)

print('Ready to talk! Type your messages here or send "exit"')

while True:
    message = input()
    if message == 'exit':
        break
    responses = agent.handle_message(message)
    for response in responses:
        print(response['text'])

Ready to talk! Type your messages here or send "exit"


 hi


Hey! How are you?


 awful


Here is something to cheer you up: https://source.unsplash.com/1600x900/?cat
Did that help you?


 yes


Great carry on!


 weather London


It's Cloudy and 10°C in London, England, GB at 07:00 AM GMT


 exit
